In [6]:
import dotenv
import polars as pl
import numpy as np
from pathlib import Path
import plotly.express as px
project_dir = Path(globals()['_dh'][0]).parent
import json
print(project_dir)
import hashlib
import pytz
from datetime import datetime, timedelta
import os
#importlib.import_module(str(project_dir/'src'/'wakatime_handler.py'))
from src.wakatime_handler import initialize_session

/home/skewballfox/Workspace/Classes/Visualization/AnatomyOfFlow


## Scratch for Wakatime

In [ ]:

state = hashlib.sha1(os.urandom(40)).hexdigest()
waka_session=initialize_session((project_dir / '.config' / 'wakatime.json'), state)

## Scratch for activity watch

In [ ]:
from aw_client import client as aw_client
from datetime import datetime, timedelta,timezone,time
import os
hostname = os.uname().nodename
client=aw_client.ActivityWatchClient()
#windows = aw_client.ActivityWatchClient(client_name="aw-watcher-window")
client.connect()
#windows.connect()


aw_window_events=client.get_events(f'aw-watcher-window_{hostname}')

aw_browser_events=client.get_events('aw-watcher-web-chrome')


In [ ]:
start=datetime.now().astimezone().replace(hour=6,minute=0,second=0, microsecond=0)
print(start.astimezone(timezone.utc))
aw_afk_events=client.get_events(f'aw-watcher-afk_{hostname}',start=start,end=start.replace(hour=17,minute=0,second=0))
aw_afk_events[-1].timestamp.astimezone().isoformat()
print(aw_afk_events)

In [ ]:
from src.user_events import build_event_df
workday_data = build_event_df(client, hostname)

# Scratchpad for Oura


In [7]:



start = (
    datetime.today()
    .astimezone(tz=pytz.timezone("America/Chicago"))
    .replace(hour=0, minute=0, second=0, microsecond=0) - timedelta(days=2)
)
end=start+timedelta(days=1)
oura_token=json.load(open(project_dir/'.config'/'oura.json'))['access_token']
from src.oura_handler import get_heartrate_data
oura_data= get_heartrate_data(oura_token, start,end)

In [8]:
oura_data

{'data': [{'bpm': 92,
   'source': 'awake',
   'timestamp': '2023-05-04T13:23:49+00:00'},
  {'bpm': 78, 'source': 'awake', 'timestamp': '2023-05-04T13:23:51+00:00'},
  {'bpm': 76, 'source': 'awake', 'timestamp': '2023-05-04T13:24:20+00:00'},
  {'bpm': 76, 'source': 'awake', 'timestamp': '2023-05-04T13:34:00+00:00'},
  {'bpm': 79, 'source': 'awake', 'timestamp': '2023-05-04T13:34:07+00:00'},
  {'bpm': 93, 'source': 'awake', 'timestamp': '2023-05-04T14:22:24+00:00'},
  {'bpm': 101, 'source': 'awake', 'timestamp': '2023-05-04T14:22:39+00:00'},
  {'bpm': 106, 'source': 'awake', 'timestamp': '2023-05-04T14:22:40+00:00'},
  {'bpm': 95, 'source': 'awake', 'timestamp': '2023-05-04T14:27:28+00:00'},
  {'bpm': 94, 'source': 'awake', 'timestamp': '2023-05-04T14:27:57+00:00'},
  {'bpm': 97, 'source': 'awake', 'timestamp': '2023-05-04T14:27:59+00:00'},
  {'bpm': 98, 'source': 'awake', 'timestamp': '2023-05-04T14:32:40+00:00'},
  {'bpm': 100, 'source': 'awake', 'timestamp': '2023-05-04T14:32:46+00:0

In [ ]:

from xlsxwriter import Workbook
with Workbook("event_df.xlsx",options={'remove_timezone':True}) as wb:
    for sheet, df in workday_data.items():
        #event_df.select(pl.col(["Start","End"]).cast(pl.Datetime).dt.replace_time_zone(None))
        #event_df.sort(by="Start", descending=False)
        df.sort(by="Start",descending=False).write_excel(workbook=wb,worksheet=sheet)

In [ ]:
from datetime import timedelta

from src.user_events import parse_terminal_history_interval
terminals={"kitty"}
browsers={"google-chrome"}
social_apps={"Ripcord","Signal"}
waka_date_current=datetime.fromisocalendar(1000,1,1).date()
df = pl.DataFrame(schema={"Category": pl.Categorical, "Start": pl.Datetime, "End": pl.Datetime,"Data":str})
waka_current_beats: dict={}
covered=terminals|browsers|social_apps
event_dict={ "Category": [], "Start": [], "End": [],"Data":[]}
for web_event in aw_window_events:
    event_start=web_event.timestamp
    event_end=event_start+web_event.duration
    if web_event.data["app"]=="code-url-handler":
        event_timestamp=web_event.timestamp
        
        #title is the file in focus, has a ● in front of it if it's unsaved
        aw_code_file:str=web_event.data["title"].removeprefix("●")
        print(aw_code_file.removeprefix("●"))
        
        if waka_date_current!=event_timestamp.date():
            waka_current_beats=dict(waka_session.get(f'users/current/heartbeats?date={event_timestamp.date().isoformat()}').json())#['data']
            print(waka_current_beats)
            waka_date_current=event_timestamp.date()
    elif web_event.data["app"] in terminals:
        commands=parse_terminal_history_interval(event_start,event_end)
        for i in range(len(commands)):
            event_dict["Category"].append("Terminal")
            event_dict["Start"].append(commands[i][0])
            event_dict["Data"].append(commands[i][1])
            if i!=len(commands)-1:
                event_dict["End"].append(commands[i+1][0])
            else:
                event_dict["End"].append(event_end)
    elif web_event.data["app"] in browsers:
        print(web_event.data["app"])
                
            
            
            
            


In [ ]:
for web_event in aw_window_events:
    if web_event['data']['app'] == 'google-chrome':
        start=web_event.timestamp #['timestamp']
        end=start+web_event.duration
        result=client.get_events('aw-watcher-web-chrome',start=start,end=end)
        if len(result)>1:
            for res in result:
                if not res.data['title'].endswith(' - Google Search'):
                    print(res.data['title'].removesuffix(' - Google Search'))
                #print(res.data)
            


In [ ]:

from subprocess import run
from src.atuin_handler import get_history_interval
atuin_env=os.environ|{'ATUIN_SESSION':str(state)}
result=[]
for web_event in aw_window_events:
    if web_event['data']['app']=="kitty":
        #directory=event['data']['title']
        start=web_event.timestamp #['timestamp']
        end=start+web_event.duration
        commands=get_history_interval(start,end)
        if commands:
            result.append(commands)
result
        

In [ ]:
from subprocess import run, PIPE
from sqlite3 import connect as sql_connect
atuin_db:Path=Path.home()/Path(".local/share/atuin/history.db")
atuin_env=os.environ|{'ATUIN_SESSION':str(state)}
run(f"atuin search --after {start.isoformat()} --before {end.isoformat()}",env=atuin_env,capture_output=True,text=True,shell=True)
#'--after',str(start.isoformat()),'--before',str(end.isoformat())],env=atuin_env,capture_output=True,text=True,shell=True)
#atuin_cursor=sql_connect(atuin_db).cursor()
#atuin_cursor.execute('select * from sqlite_schema')
#atuin_cursor.fetchall()

In [ ]:
project_dir

In [ ]:
from oura import OuraClient
oura_config = dict(json.load(open(project_dir / '.config' / 'oura.json')))
oura_client=OuraClient(personal_access_token=oura_config['access_token'])
# .sleep_summary(start=datetime(2023, 3, 18).isoformat(), end=datetime(2023, 3, 19).isoformat())
#OuraClient(personal_access_token=oura_config['access_token'])
oura_client.